In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
plt.rcParams.update({'font.size': 22})

### Evaluation criterions

In this notebook 2 different traversability models are compared for trajectory optimization:

* geometrical (`map_rpz`),
* learned (`learned_rpz`).

The following metrics are calculated in `Cave` environments for `Naex + Trajectory optimization (Map RPZ)` and `Naex + Trajectory optimization (Learned RPZ)`.

In [ ]:
common_metrics = [
    'Exploration face loss',
#     'Exploration edge loss',
#     'Exploration chamfer loss',
    'Exploration completeness',
#     'Map face loss',
#     'Map edge loss',
#     'Map chamfer loss',
    'Artifacts exploration completeness',
    'Detections score',
    'mAP',
    'Total expected reward',
    'Total artifacts reward',
    'Travelled distance',
    'Total actual reward',
#     'Localization error: pos',
#     'Localization error: ang'
#     'Artifacts localization error',
]

worlds = ["simple_cave_01", "simple_cave_02", "simple_cave_03"]
# worlds = ["finals_qual"]

MIN_EXP_COMPL = 0.05
MIN_TRAVELLED_DIST = 100

paths = ['../data/metrics/naex/trajopt_trav_compare/']

# all metrics files
xls_files = [os.path.join(path, file) for path in paths for file in os.listdir(path) if 'xls' in file]
    
# filter by worlds
xls_files1 = []
for file in xls_files:
    for world in worlds:
        if world in file:
            xls_files1.append(file)
xls_files = xls_files1
    
# filter by computers
computers = ['ritz', 'halmos', 'goedel']
# computers = ['halmos', 'ritz', 'zorn']
# computers = ['halmos', 'ritz']
# computers = ['ritz']

xls_files1 = []
for file in xls_files:
    for comp in computers:
        if comp in file:
            xls_files1.append(file)
xls_files = xls_files1

In [ ]:
# names = [
#     'Time stamp',
#     'Exploration face loss',
#     'Exploration edge loss',
#     'Exploration chamfer loss',
#     'Exploration completeness',
#     'Map face loss',
#     'Map edge loss',
#     'Map chamfer loss',
#     'Artifacts exploration completeness',
#     'Detections score',
#     'mAP',
#     'Total expected reward',
#     'Total artifacts reward',
#     'Travelled distance',
#     'Total actual reward',
#     'Localization error: pos',
#     'Localization error: ang'
#     'Artifacts localization error',
# ]

# from xlrd import open_workbook
# from xlutils.copy import copy

# for filepath in xls_files:
#     rb = open_workbook(filepath)
#     wb = copy(rb)
#     s = wb.get_sheet(0)
#     for i, name in enumerate(names):
#         s.write(0, i, name)
#     wb.save(filepath)

### All experiments

In [ ]:
for world in worlds:
    for i, metric in enumerate(common_metrics):
        plt.figure(figsize=(10, 7))
        plt.title(f"World: {world}, metric: {metric}")
        xls_files_world = [file for file in xls_files if world in file]
        for file in xls_files_world:
            data = pd.read_excel(file)
            data = data.fillna(method='ffill')

            if metric in data and \
               np.max(data['Localization error: pos']) < 5.0 and \
               np.max(data['Exploration completeness']) > MIN_EXP_COMPL and \
               np.max(data['Travelled distance']) > MIN_TRAVELLED_DIST:
                
                ls = 'solid' if 'map_rpz' in file else 'dashed'
                t = data['Time stamp'] - data['Time stamp'][0]
                plt.plot(t, data[metric], linestyle=ls, label=file)
                                
        plt.grid()
        plt.xlabel('time, [sec]')
        # plt.legend()
        plt.show()
        plt.close()

In [ ]:
for key in data:
    print(key)

In [ ]:
data.head()

### Average results

In [ ]:
xls_files_useful = []

for metric in common_metrics:
    ts, values = [], []
    ts_maprpz, values_maprpz = [], []
    for file in xls_files:
        data = pd.read_excel(file)
        data = data.fillna(method='ffill')
        
        if metric in data and \
           np.max(data['Localization error: pos'][0:5]) < 5.0 and \
           np.max(data['Exploration completeness']) > MIN_EXP_COMPL and \
           np.max(data['Travelled distance']) > MIN_TRAVELLED_DIST:
                  
            t = np.array(data["Time stamp"] - data["Time stamp"][0])
            value = np.array(data[metric])

            if 'map_rpz' in file:
                ts_maprpz.append(t)
                values_maprpz.append(value)
            else:
                ts.append(t)
                values.append(value)

            xls_files_useful.append(file)

    if len(xls_files_useful) == 0:
        print('No data')
        break

    t_mean = np.linspace(0, 3600, 37)

    values_interp = [np.interp(t_mean, t, v) for t, v in zip(ts, values)]
    mean = np.mean(values_interp, axis=0) 
    std = np.std(values_interp, axis=0)
    
    values_interp = [np.interp(t_mean, t, v) for t, v in zip(ts_maprpz, values_maprpz)]
    mean_maprpz = np.mean(values_interp, axis=0) 
    std_maprpz = np.std(values_interp, axis=0)

    plt.figure(figsize=(10, 7))
    plt.title(metric)
    # for t, v, v_interp in zip(ts, values, values_interp):
    #     plt.plot(t, v)
    #     plt.plot(t_mean, v_interp, 'x')

    plt.plot(t_mean, mean, label='naex_trajopt_learned_rpz', color='orange')
    plt.fill_between(t_mean, mean-std, mean+std, color='orange', alpha=0.2)
    
    plt.plot(t_mean, mean_maprpz, label='naex_trajopt_map_rpz', color='green')
    plt.fill_between(t_mean, mean_maprpz-std_maprpz, mean_maprpz+std_maprpz, color='green', alpha=0.2)
    
    plt.grid()
    plt.xlabel('time, [sec]')
    plt.legend()
    
    # save results
    if not os.path.exists('./results/naex_trajopt_trav_compare/'):
        os.mkdir('./results/naex_trajopt_trav_compare/')
    plt.savefig('./results/naex_trajopt_trav_compare/' + metric + '.png')

    plt.show()
    plt.close()

In [ ]:
xls_files_useful = set(xls_files_useful)  # remove dublicates

for world_name in worlds:
    n_map_rpz = len([file for file in xls_files_useful if world_name in file and "map_rpz" in file])
    n_learned_rpz = len([file for file in xls_files_useful if world_name in file and not "map_rpz" in file])

    print(world_name, "N map rpz:", n_map_rpz, "N learned rpz:", n_learned_rpz)

In [ ]:
for cmp_name in computers:
    n_map_rpz = len([file for file in xls_files_useful if cmp_name in file and "map_rpz" in file])
    n_learned_rpz = len([file for file in xls_files_useful if cmp_name in file and not "map_rpz" in file])

    print(cmp_name, "N map rpz:", n_map_rpz, "N learned rpz:", n_learned_rpz)